# Домашнее задание 4. Реализация MinHashLSH.

## Алгоритм MinHash. https://en.wikipedia.org/wiki/MinHash

1) Разбиваем текст на токены/шинглы \
2) Берем хеш функцию от каждого шингла \
3) Выбираем для каждого документа первый минимальный хеш (не пустой) и индекс этого элемента записываем в матрицу сигнатур \
4) Получаем матрицу сигнатур: премешиваем строки -> первый минимальный хеш может измениться -> выбираем (повторяем это N раз) \
5) Считаем расстояние жаккарда по полученной матрице

## пример:

возьмем 3 документа: 
- Doc1 = 'я люблю Париж' 
- Doc2 = 'я не люблю Париж' 
- Doc3 = 'я люблю Рим' 


1) посплитим их на слова (тут можно усложнять, брать шинглы, обученный токенезатор и т.д.) \

- Doc1 = ['я', 'люблю', 'Париж'] 
- Doc2 = ['я', 'не люблю', 'Париж'] 
- Doc3 = ['я', 'люблю', 'Рим']

2) составим матрицу встречаемости каждого из слов

картинка


3) Перемешаем индексы (чтобы перемешивать - можно использовать хеш функцию, которую применяем к индексу)

картинка

для первого перемешивания: \
[3, 1, 2] \
для второго перемешивания: \
[2, 1, 2] \
для третьего: \
[2, 1, 3]


4) получаем матрицу сигнатур \
    Doc1 Doc2 Doc3  \
    [3,   1,    2]  \
    [2,   1,    2]  \
    [2,   1,    3] 

5) Расстояние Жаккарда:

          Doc1 Doc2 Doc3 
    Doc1    1    0   1/2 
    Doc2    0    1    0  
    Doc3    1/2  0    1  
    
    
Получаем, что 1 и 3 документ хоть как-то похожи, а больше похожих пар нет.

In [4]:
import pandas as pd

In [5]:
df = pd.read_parquet('train-00013-of-00041.parquet')

In [7]:
df['text'][:10]

0    The Frances Ellen Watkins Harper House is a hi...
1    Uza may refer to:\nPlaces\nUza, Landes, a vill...
2    Tommy Kirkham is a Northern Ireland loyalist p...
3    A cotyledon is a significant part of the embry...
4    Det som varit ÄR is a Viking rock album, relea...
5    Middlebury Township may refer to the following...
6    The list of North Carolina hurricanes between ...
7    The Martyrs of Abitinae (or Abitinian Martyrs)...
8    The Fabyan Windmill is an authentic, working D...
9    HLA-B54 (B54) is an HLA-B serotype. B54 is a s...
Name: text, dtype: object

In [288]:
import re

class MinHash:
    def __init__(self, num_permutations: int):
        self.num_permutations = num_permutations

    def preprocess_text(self, text: str) -> str:
        return re.sub("( )+|(\n)+"," ",text).lower()

    def tokenize(self, text: str) -> set:
        text = self.preprocess_text(text)      
        return set(text.split(' '))
    
    def get_occurrence_matrix(self, corpus_of_texts: list[set]) -> pd.DataFrame:
        occurrence_dict = {}
        for i,x in enumerate(corpus_of_texts):
            tokens = self.tokenize(x)
            occurrence_dict[i] = {token: 1 for token in tokens}
        
        return pd.DataFrame(occurrence_dict).fillna(0.0)
    
    def get_new_index(self, x: int, permutation_index: int) -> int:
        ### values_dict - нужен для совпадения результатов теста, а в общем случае используется рандом
        values_dict = {
            'a': [3, 4, 5, 7, 8],
            'b': [3, 4, 5, 7, 8] 
        }
        a = values_dict['a'][permutation_index]
        b = values_dict['b'][permutation_index]
        return (a*(x+1) + b) % 23 ### здесь важно, чтобы число было >= rows_number и было боижайшим простым числом к rows_number
    
    
    def _get_jaccard_similarity(self, set_a: set, set_b: set) -> float:
        intersection = set_a & set_b
        union = set_a | set_b
        return len(intersection) / len(union)
    
    def get_jaccard_similarity(self, min_hash_matrix) -> float:
        jaccard_matrix = np.zeros((min_hash_matrix.shape[1], min_hash_matrix.shape[1]))
        for i in range(0, min_hash_matrix.shape[1]):
            for j in range(i+1, min_hash_matrix.shape[1]):
                jaccard_matrix[i][j] = self._get_jaccard_similarity(set(ar[::,i]), set(ar[::,j]))
        return jaccard_matrix
     
    
    def get_minhash(self, occurrence_matrix: pd.DataFrame) -> np.array:
        min_hash_matrix = np.zeros((self.num_permutations, len(occurrence_matrix.columns)))
        num_docs = len(occurrence_matrix.columns)
        docs_names = occurrence_matrix.columns
        for num in range(self.num_permutations):
            occurrence_matrix[f'new_index_{i}'] = [self.get_new_index(index, permutation_index=num) for index in range(len(occurrence_matrix))]
            for doc_id, doc_name in enumerate(docs_names):
                min_hash_matrix[num][doc_id] = occurrence_matrix[occurrence_matrix[doc_id] == 1][f'new_index_{i}'].min()
            
        return min_hash_matrix
    
    def run_minhash(self,  corpus_of_texts: list[str]):
        occurrence_matrix = self.get_occurrence_matrix(corpus_of_texts)
        minhash = self.get_minhash(occurrence_matrix)
        jaccard_matrix = self.get_jaccard_similarity(minhash)
        return jaccard_matrix
    
    
    

In [240]:
minhash = MinHash(3)

In [241]:
matrix = minhash.run(Docs)

я люблю Париж
я не люблю Париж
я люблю Рим
         0    1    2
люблю  1.0  1.0  1.0
париж  1.0  1.0  0.0
я      1.0  1.0  1.0
не     0.0  1.0  0.0
рим    0.0  0.0  1.0





In [242]:
matrix

array([[0.        , 1.        , 0.33333333],
       [0.        , 0.        , 0.33333333],
       [0.        , 0.        , 0.        ]])

In [187]:
Docs = ['я люблю Париж', 
'я не люблю Париж' ,
'я люблю Рим']

## Алгоритм MinHashLSH. 

Одна из самых дорогих операций - это попарное сравнения похожести двух объектов. Давайте разобьем матрицу сигнатур на N батчей и будем сравнивать документы внутри небольших батчей. Если сигнатуры документа А и B полностью совпадают - то документы похожи. То есть если совпали хотя бы в одном бакете - документы похожи, если нет - то считаем что нет. Далее к оставшимся документам  уже можно применить полноценный MinHash без батчей.


Возьмем матрицу сигнатур из первого примера:


    Doc1 Doc2 Doc3  
    [3,   1,    2]  
    [2,   1,    2]  
    [2,   1,    3] 
    [1,   1,    1]
    
    
И разобем ее на 2 бакета:



    Doc1 Doc2 Doc3  
    [3,   1,    2]  b1
    [2,   1,    2]  b1
    [2,   1,    3]  b2
    [1,   1,    1]  b2
    
    
Если сравнивать по бакетам - то кандидатами на похожие являются только документы Doc2, Doc3.


В задании для простоты, давайте считать что баскеты будем распределять по порядку как на примере.

In [299]:
class MinHashLSH(MinHash):
    def __init__(self, num_permutations: int, num_buckets: int):
        self.num_permutations = num_permutations
        self.num_buckets = num_buckets
        
    def get_buckets(self, minhash: np.array) -> np.array:
        size_of_bucket = int(len(minhash)/self.num_buckets)
        
        if len(minhash) <= size_of_bucket:
            return np.array([minhash])
        
        res_array = []
        for i in range(min(self.num_buckets, len(minhash))):
            res_array.append(minhash[i*size_of_bucket:(i+1)*size_of_bucket])
        
        return np.array(res_array)
    
    def get_similar_candidates(self, buckets) -> list[tuple]:
        similar_candidates = []
        for bucket in buckets:
            for i in range(bucket.shape[1]):
                for j in range(i+1, bucket.shape[1]):
                    if (bucket[::, i] == bucket[::, j]).all():
                        similar_candidates.append((i,j))
        return similar_candidates
        
    def run_minhash_lsh(self, corpus_of_texts: list[str]) -> list[tuple]:
        occurrence_matrix = self.get_occurrence_matrix(corpus_of_texts)
        minhash = self.get_minhash(occurrence_matrix)
        buckets = self.get_buckets(minhash)
        similar_candidates = self.get_similar_candidates(buckets)
        
        return set(similar_candidates)
    

In [300]:
min_hash_lsh = MinHashLSH(5,10)

In [301]:
min_hash_lsh.run_minhash_lsh(Docs)

{(0, 1), (0, 2), (1, 2)}